In [9]:
import numpy as np
import os
import cv2
import shutil
import random as rn
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
data_dir ="/content/drive/MyDrive/IBM/flowers"

In [12]:
print(os.listdir("/content/drive/MyDrive/IBM/flowers"))

['dandelion', 'tulip', 'daisy', 'sunflower', 'rose']


In [13]:
batch_size = 32
img_height = 180
img_width = 180
num_classes = 5

In [14]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4317 files belonging to 5 classes.
Using 3454 files for training.


In [15]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4317 files belonging to 5 classes.
Using 863 files for validation.


In [16]:
class_names = train_ds.class_names
class_names

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [17]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [18]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [19]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0



* ### ***Image Augmentation***





In [20]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

*   ### ***Model Creation / Adding Layers***

In [21]:
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.3),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dense(num_classes)
])

* ### ***Compling the Model***

In [22]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

* ### ***Fitting the Model***

In [23]:
epochs=25
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/25
108/108 [==============================] - 94s 781ms/step - loss: 1.3248 - accuracy: 0.4134 - val_loss: 1.1138 - val_accuracy: 0.5608
Epoch 2/25
108/108 [==============================] - 3s 29ms/step - loss: 1.0422 - accuracy: 0.5692 - val_loss: 1.0106 - val_accuracy: 0.5968
Epoch 3/25
108/108 [==============================] - 3s 29ms/step - loss: 0.9695 - accuracy: 0.6141 - val_loss: 0.9223 - val_accuracy: 0.6362
Epoch 4/25
108/108 [==============================] - 3s 29ms/step - loss: 0.8898 - accuracy: 0.6477 - val_loss: 0.8969 - val_accuracy: 0.6501
Epoch 5/25
108/108 [==============================] - 3s 30ms/step - loss: 0.8404 - accuracy: 0.6760 - val_loss: 0.8947 - val_accuracy: 0.6454
Epoch 6/25
108/108 [==============================] - 3s 29ms/step - loss: 0.8274 - accuracy: 0.6749 - val_loss: 0.8415 - val_accuracy: 0.6906
Epoch 7/25
108/108 [==============================] - 3s 29ms/step - loss: 0.7662 - accuracy: 0.7061 - val_loss: 0.7691 - val_accuracy: 0.71

* ### ***Testing on unseen image Data***

In [25]:
from matplotlib import image as im
from matplotlib import pyplot
from keras.preprocessing import image
from PIL import Image
data = im.imread('/content/drive/MyDrive/IBM/flowers/daisy/100080576_f52e8ee070_n.jpg')
img=image.load_img('/content/drive/MyDrive/IBM/flowers/daisy/100080576_f52e8ee070_n.jpg', target_size=(180, 180))

test_img=np.expand_dims(img , axis=0)
  
result = model.predict(test_img)
pred = np.argmax(result) 
print(result)
print(pred)
print(class_names)



[[ 6.6757097  2.4115148 -3.093146  -1.7972815 -4.8765335]]
0
['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [26]:
image = tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/IBM/flowers/daisy/100080576_f52e8ee070_n.jpg', target_size=(180,180))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  
result = model.predict(input_arr)
pred = np.argmax(result) 
print(class_names)
print(pred)
print(class_names[pred])

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
0
daisy


* ### ***Saving Model***

In [27]:
model.save("/content/drive/MyDrive/IBM/flower_model.h5")

In [28]:
from tensorflow.keras.models import load_model

savedModel=load_model("/content/drive/MyDrive/IBM/flower_model.h5")
savedModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                  